In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Data import and basic summary
np.random.seed(49)
DashData = pd.read_csv("C:/Users/Zach/Documents/GitHub/DoorDashModels/Data/historical_data.csv")
DashData.head(10)
DashData.info()
DashData.describe()
print("Percentange of missing value % :",DashData.isna().mean().mean()*100)

In [ ]:
#Creating the target variable for models
from datetime import datetime
DashData['created_at'] = pd.to_datetime(DashData['created_at'])
DashData["actual_delivery_time"] = pd.to_datetime(DashData["actual_delivery_time"])
DashData["Total_Delivery_Duration"] = (DashData["actual_delivery_time"] - DashData["created_at"]).dt.total_seconds()

DashData.head(10)
#New feature creations
DashData['Estimated_non_prep_duration'] = DashData["estimated_store_to_consumer_driving_duration"] + DashData["estimated_order_place_duration"]
DashData['BusyDriverRatio'] = DashData["total_busy_dashers"] / DashData["total_onshift_dashers"]
#Time features
DashData['Months'] = DashData['created_at'].dt.month
DashData['Quarter'] = DashData['created_at'].dt.quarter
#aggregate features
DashData['TotalOrderPerStore'] = DashData.groupby('store_id')['total_items'].transform('sum')
DashData['OrderValuePerStore'] = DashData.groupby('store_id')['total_items'].transform('mean')
DashData['OrderCountMonthsPerStore'] = DashData.groupby('Months')['total_items'].transform('sum')

#simple store metrics
avg_by_category= DashData.groupby('store_primary_category')['subtotal'].mean() / 100
print("Average subtotals per store category in dollars:")
print(avg_by_category)
std_by_category = DashData.groupby('store_primary_category')['subtotal'].std() / 100
print("\nStandard deviation of subtotals per store category in dollars:")
print(std_by_category)
print("Average order delivery time in minutes:")
print(f"{DashData['Total_Delivery_Duration'].mean()/60:.2f}")

DashData.head()



In [71]:
#creating a dictionary for store categories to fill the null categories using the most common category -- could be better way to do with with SME
store_id_unique = DashData["store_id"].unique().tolist()
store_id_and_category = {store_id: DashData[DashData.store_id == store_id].store_primary_category.mode() 
                         for store_id in store_id_unique}
def fill(store_id):
    """Return primary store category from the dictionary"""
    try:
        return store_id_and_category[store_id].values[0]
    except:
        return np.nan

# fill null values
DashData["nan_free_store_primary_category"] = DashData.store_id.apply(fill)


In [ ]:
#creating dummies
OrderProtocalDummies =pd.get_dummies(DashData.order_protocol, dtype=int)
OrderProtocalDummies = OrderProtocalDummies.add_prefix('OrderProtocal_')
#checking
OrderProtocalDummies.head()
MarketIdDummies = pd.get_dummies(DashData.market_id, dtype=int)
MarketIdDummies = MarketIdDummies.add_prefix('MarketId_')
#checking
MarketIdDummies.head()
StoreCategoryDummies = pd.get_dummies(DashData.nan_free_store_primary_category, dtype=int)
StoreCategoryDummies = StoreCategoryDummies.add_prefix('StoreCategory_')
#checking
StoreCategoryDummies.head()

In [93]:
# defining the intial train DF
train_df = DashData.drop(columns = ["created_at", "market_id", "store_id", "store_primary_category", "actual_delivery_time","nan_free_store_primary_category", "order_protocol"])
train_df.head()
#merging the dummies to train
train_df = pd.concat([train_df, OrderProtocalDummies, MarketIdDummies, StoreCategoryDummies], axis=1)
#checking
train_df.head()
#seeing which fetures that have infinte max value -- could be issue for models
if np.isinf(train_df).any().any():
    print("Features with infinite max value:")
    print(train_df.columns[np.isinf(train_df).any()])
else:
    print("No infinite values in the dataset")
#ensuring dtype synergy
train_df = train_df.astype("Float32")

Features with infinite max value:
Index(['BusyDriverRatio'], dtype='object')


In [95]:
#replacing infinite values in BusyDriverRatio with NaN then with the mean of the column
train_df["BusyDriverRatio"] = train_df["BusyDriverRatio"].replace([np.inf, -np.inf], np.nan)
train_df["BusyDriverRatio"] = train_df["BusyDriverRatio"].fillna(train_df["BusyDriverRatio"].mean())
#checking
train_df["BusyDriverRatio"].describe()


count    197428.0
mean     0.949777
std      0.385194
min         -13.0
25%      0.846154
50%      0.949778
75%           1.0
max          31.0
Name: BusyDriverRatio, dtype: Float64

In [ ]:
#Next Task (3/20): Removing Redundant and Potential Collinear Features